# Load the Data & Train on a small subset

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from modules.data import SessionDataset

PATH_HOME = Path.home()
PATH_PROJ = PATH_HOME/'pyGRU4REC' 
PATH_DATA = PATH_PROJ/'data'
PATH_MODEL = PATH_PROJ/'models'
train = 'train.tsv'
test = 'test.tsv'
PATH_TRAIN = PATH_DATA / train
PATH_TEST = PATH_DATA / test
n_samples = -1
n_samples = 100000
train_dataset = SessionDataset(PATH_TRAIN, n_samples=n_samples)
test_dataset = SessionDataset(PATH_TEST, n_samples=n_samples, itemmap=train_dataset.itemmap)

Train on a small subset of data

In [2]:
from modules.model import GRU4REC
import torch

input_size = len(train_dataset.items)
hidden_size = 100
num_layers = 1
output_size = input_size
batch_size = 50

optimizer_type = 'Adagrad'
lr = .01
weight_decay = 0
momentum = 0
eps = 1e-6

loss_type = 'TOP1'

n_epochs = 10
use_cuda = True

torch.manual_seed(7)

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers=num_layers,
                batch_size=batch_size,
                optimizer_type=optimizer_type,
                lr=lr,
                weight_decay=weight_decay,
                momentum=momentum,
                eps=eps,
                loss_type=loss_type,
                use_cuda=use_cuda)

model_name = 'GRU4REC'
model.train(train_dataset, n_epochs=n_epochs, model_name=model_name, save=False)
model.test(test_dataset)

/usr/local/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Training GRU4REC...
epoch: 1/loss:0.956/recall:0.272/mrr:0.154/time:0.059
epoch: 2/loss:0.909/recall:0.445/mrr:0.244/time:0.058
epoch: 3/loss:0.886/recall:0.517/mrr:0.279/time:0.058
epoch: 4/loss:0.872/recall:0.568/mrr:0.306/time:0.058
epoch: 5/loss:0.862/recall:0.607/mrr:0.329/time:0.058
epoch: 6/loss:0.855/recall:0.635/mrr:0.348/time:0.058
epoch: 7/loss:0.849/recall:0.657/mrr:0.364/time:0.058
epoch: 8/loss:0.844/recall:0.676/mrr:0.375/time:0.058
epoch: 9/loss:0.839/recall:0.691/mrr:0.385/time:0.058
epoch:10/loss:0.836/recall:0.704/mrr:0.394/time:0.058
Test result: loss:0.979/recall:0.088/mrr:0.046/time:0.003


# Full Training

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from modules.data import SessionDataset
from modules.model import GRU4REC


PATH_HOME = Path.home()
PATH_PROJ = PATH_HOME/'pyGRU4REC' 
PATH_DATA = PATH_PROJ/'data'
PATH_MODEL = PATH_PROJ/'models'
train = 'train.tsv'
test = 'test.tsv'
PATH_TRAIN = PATH_DATA / train
PATH_TEST = PATH_DATA / test

train_dataset = SessionDataset(PATH_TRAIN)
test_dataset = SessionDataset(PATH_TEST, itemmap=train_dataset.itemmap)

input_size = len(train_dataset.items)
hidden_size = 100
num_layers = 1
output_size = input_size
batch_size = 50

optimizer_type = 'Adagrad'
lr = .01
weight_decay = 0
momentum = 0
eps = 1e-6

loss_type = 'TOP1'

n_epochs = 10
use_cuda = True

torch.manual_seed(7)

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers=num_layers,
                batch_size=batch_size,
                optimizer_type=optimizer_type,
                lr=lr,
                weight_decay=weight_decay,
                momentum=momentum,
                eps=eps,
                loss_type=loss_type,
                use_cuda=use_cuda)

model_name = 'GRU4REC'
model.train(train_dataset, n_epochs=n_epochs, model_name=model_name, save=False)
model.test(test_dataset)

## Evaluate using the trained models

In [3]:
from pathlib import Path
from modules.data import SessionDataset

PATH_HOME = Path.home()
PATH_PROJ = PATH_HOME/'pyGRU4REC' 
PATH_DATA = PATH_PROJ/'data'
PATH_MODEL = PATH_PROJ/'models'
train = 'train.tsv'
test = 'test.tsv'
PATH_TRAIN = PATH_DATA / train
PATH_TEST = PATH_DATA / test
train_dataset = SessionDataset(PATH_TRAIN)
test_dataset = SessionDataset(PATH_TEST, itemmap=train_dataset.itemmap)

## 1.Load the Common Parameters

In [4]:
import torch
from modules.layer import GRU
from modules.model import GRU4REC

input_size = len(train_dataset.items)
output_size = input_size
hidden_size = 100
num_layers = 1

use_cuda = True
time_sort = False

optimizer_type = 'Adagrad'

## 2. Evaluation(TOP1 Loss)

In [5]:
model_name = 'GRU4REC_TOP1_Adagrad_0.01_epoch5'
model_file = PATH_MODEL/model_name

loss_type = 'TOP1'
lr = 0.01

dropout_hidden = .5
dropout_input = 0
batch_size = 50
momentum = 0

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = dropout_input,
                dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = loss_type,
                optimizer_type = optimizer_type,
                lr=lr,
                momentum=momentum,
                time_sort=time_sort,
                pretrained=gru)

k = 20
model.test(test_dataset, k=k)

/usr/local/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Test result: loss:0.857/recall:0.628/mrr:0.264/time:0.033


## 3. Evaluation(BPR Loss)

In [6]:
model_name = 'GRU4REC_BPR_Adagrad_0.05_epoch5'
model_file = PATH_MODEL/model_name

loss_type = 'BPR'
lr = 0.05

dropout_hidden = .2
dropout_input = 0
batch_size = 50
momentum = 0.2

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = dropout_input,
                dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = loss_type,
                optimizer_type = optimizer_type,
                lr=lr,
                momentum=momentum,
                time_sort=time_sort,
                pretrained=gru)

k = 20
model.test(test_dataset, k=k)

/usr/local/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Test result: loss:0.289/recall:0.599/mrr:0.239/time:0.032


## 4. Evaluation(CrossEntropyLoss)

In [7]:
model_name = 'GRU4REC_CrossEntropy_Adagrad_0.01_epoch5'
model_file = PATH_MODEL/model_name

loss_type = 'CrossEntropy'
lr = 0.01

dropout_hidden = 0
dropout_input = 0
batch_size = 500
momentum = 0

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = dropout_input,
                dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = loss_type,
                optimizer_type = optimizer_type,
                lr=lr,
                momentum=momentum,
                time_sort=time_sort,
                pretrained=gru)

k = 20
model.test(test_dataset, k=k)

Test result: loss:5.051/recall:0.620/mrr:0.254/time:0.013
